<a href="https://colab.research.google.com/github/edinaldoab/previsao_remuneracao/blob/master/tratamento/tratamento_dos_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etapa 02: Preparação do dataset para os modelos de ML



---



In [100]:
!rm -rf previsao_remuneracao/
!git clone https://github.com/edinaldoab/previsao_remuneracao.git

Cloning into 'previsao_remuneracao'...
remote: Enumerating objects: 32065, done.
remote: Counting objects: 100% (1351/1351), done.
remote: Compressing objects: 100% (979/979), done.
remote: Total 32065 (delta 340), reused 1330 (delta 333), pack-reused 30714
Receiving objects: 100% (32065/32065), 432.41 MiB | 23.92 MiB/s, done.
Resolving deltas: 100% (6882/6882), done.
Updating files: 100% (32855/32855), done.


## Instalação do PySpark

In [4]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [6]:
import findspark
findspark.init()

### [SparkSession](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.html)

O ponto de entrada para programar o Spark com a API Dataset e DataFrame.

Uma SparkSession pode ser utilizada para criar DataFrames, registrar DataFrames como tabelas, executar consultas SQL em tabelas, armazenar em cache e ler arquivos parquet. Para criar uma SparkSession, use o seguinte padrão de construtor:

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Tratamento dos dados de vagas do GD") \
    .getOrCreate()

spark

## Carregamento dos Dados

## Leitura do dataset

In [8]:
url_github = 'https://raw.githubusercontent.com/edinaldoab/previsao_remuneracao/master/notebook_scrapping/Data_Engineer_United_States.csv'

In [9]:
from pyspark import SparkFiles

spark.sparkContext.addFile(url_github)

dados = spark.read.csv(SparkFiles.get('Data_Engineer_United_States.csv'), header=True, multiLine=True, escape='"')

In [10]:
dados.count()

450

In [11]:
dados.limit(5).toPandas()

,company,job title,location,job description,salary estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue
0,DataPattern,Sr. Data Engineer,"Los Angeles, CA",Responsibilities\n● Contribute to the design a...,$65.00 - $75.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable
1,YT Global Network,Data Engineer- Remote,Remote,Data Engineer- Remote\nRole: Data and Analytic...,$90.00 - $120.00 Per hour,Unknown,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable
2,Wevision LLC,Data Engineer,"Irvine, CA","Job description\nWe build services, data platf...",$55.00 - $85.00 Per hour,1 to 50 Employees,Company - Private,#N/A,#N/A,#N/A,Unknown / Non-Applicable
3,kairos technologies,Data Engineer,"Boston, MA",Job Title: Data Engineer with AWS – Hybrid\nLo...,$60.00 - $68.00 Per hour,51 to 200 Employees,Company - Private,Information Technology,Information Technology Support Services,2003,$5 to $25 million (USD)
4,Konnectingtree,Data Engineer,Remote,Greetings from KonnectingTree!\nWe are looking...,$45.00 - $50.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable


## Análise e tratamento de Dados

Renomeando as colunas:

In [12]:
from pyspark.sql.functions import col

In [13]:
dados = dados\
    .select(*[col(c).alias(c.replace(' ', '_')) for c in dados.columns])

In [14]:
dados.show(3)

+-----------------+--------------------+---------------+--------------------+--------------------+-----------------+-----------------+--------------+----------------+---------------+--------------------+
|          company|           job_title|       location|     job_description|     salary_estimate|     company_size|     company_type|company_sector|company_industry|company_founded|     company_revenue|
+-----------------+--------------------+---------------+--------------------+--------------------+-----------------+-----------------+--------------+----------------+---------------+--------------------+
|      DataPattern|   Sr. Data Engineer|Los Angeles, CA|Responsibilities
...|$65.00 - $75.00 P...|1 to 50 Employees| Company - Public|          #N/A|            #N/A|           #N/A|Unknown / Non-App...|
|YT Global Network|Data Engineer- Re...|         Remote|Data Engineer- Re...|$90.00 - $120.00 ...|          Unknown| Company - Public|          #N/A|            #N/A|           #N/A|Un

In [15]:
dados.printSchema()

root
 |-- company: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- location: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- salary_estimate: string (nullable = true)
 |-- company_size: string (nullable = true)
 |-- company_type: string (nullable = true)
 |-- company_sector: string (nullable = true)
 |-- company_industry: string (nullable = true)
 |-- company_founded: string (nullable = true)
 |-- company_revenue: string (nullable = true)



### Análise e tratamento de `salary_estimate`:

In [16]:
from pyspark.sql.functions import desc

In [17]:
dados\
    .select('salary_estimate')\
    .groupBy('salary_estimate')\
    .count()\
    .sort(desc('count'))\
    .show(200, truncate=False)

+--------------------------+-----+
|salary_estimate           |count|
+--------------------------+-----+
|$1L - $2L                 |52   |
|#N/A                      |47   |
|$65.00 - $75.00 Per hour  |20   |
|$90T - $1L                |19   |
|$85T - $1L                |15   |
|$95T - $1L                |14   |
|$42.96 - $60.84 Per hour  |13   |
|$73T - $1L                |12   |
|$86T - $1L                |12   |
|$70.00 - $80.00 Per hour  |12   |
|$1L - $1L                 |12   |
|$83T - $1L                |11   |
|$60.00 - $70.00 Per hour  |11   |
|$88T - $1L                |11   |
|$43.82 - $66.67 Per hour  |11   |
|$2L - $2L                 |11   |
|$60.00 - $65.00 Per hour  |10   |
|$73.00 Per hour           |10   |
|$65.00 Per hour           |9    |
|$80.00 - $95.00 Per hour  |9    |
|$55.00 - $60.00 Per hour  |8    |
|$70.00 Per hour           |8    |
|$60.00 Per hour           |7    |
|$87T - $1L                |7    |
|$70T - $75T               |7    |
|$100.00 - $125.00 P

Eliminando vagas sem informações de salários:

In [18]:
dados = dados.filter(dados['salary_estimate'] != '#N/A')

In [19]:
dados.count()

403

Como pretende-se ter como produto uma **previsão de salário**, todos os tratamentos a seguir se dão com o objetivo de calcular uma média de estimativa, seguindo:

1.   "Explosão" do intervalo `'salary_estimate'` para se obter mínimo e máximo possível
2.   Limpeza dos caracteres de `$`
3.   Multiplicação dos valores pelos fatores L e T (padrão indiano) quando existirem
4.   Adequação dos valores `"Per hour"` para `"Per year"`
5.   Cálculo da média simples entre os valores

In [20]:
from pyspark.sql.functions import split, size

In [21]:
dados\
    .withColumn('split_size', size(split(dados['salary_estimate'],' ')))\
    .select('split_size')\
    .groupBy('split_size')\
    .count()\
    .sort(desc('count'))\
    .show(100)

+----------+-----+
|split_size|count|
+----------+-----+
|         3|  262|
|         5|  135|
|         1|    6|
+----------+-----+



Ao explodir o intervalo de salários, surgem duas possibilidades para os splits que geraram `split_size == 3`:

> *   **Valor fixo**
        Ex.: '$60 per hour' -> ['$60','per','hour']

> Neste caso, opta-se em atribuir o mesmo valor mínimo ao valor máximo, para não se ter impacto no cálculo futuro do salário médio.

> *   **com valor mínimo e com valor máximo**
        Ex.: '$75T - $1L' -> ['$75T','-','$1L']

In [22]:
dados = dados\
    .select('*')\
    .withColumn('split_size', size(split(dados['salary_estimate'], ' ')))

In [23]:
dados.printSchema()

root
 |-- company: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- location: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- salary_estimate: string (nullable = true)
 |-- company_size: string (nullable = true)
 |-- company_type: string (nullable = true)
 |-- company_sector: string (nullable = true)
 |-- company_industry: string (nullable = true)
 |-- company_founded: string (nullable = true)
 |-- company_revenue: string (nullable = true)
 |-- split_size: integer (nullable = false)



In [24]:
from pyspark.sql.functions import when, lit,concat

In [25]:
dados.withColumn('salary_estimate', 
                 when(dados['split_size'] == 1, 
                      concat(dados['salary_estimate'], lit(' - '), dados['salary_estimate'])
                     ).otherwise(dados['salary_estimate'])
                )\
                .filter(dados['split_size'] == 1)\
                .toPandas()

,company,job_title,location,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,split_size
0,VirginPulse,Data Engineer I,Remote,Overview:\nNow is the time to join us!\nAt Vir...,$80T - $80T,1001 to 5000 Employees,Company - Private,Information Technology,Computer Hardware Development,2004,$25 to $50 million (USD),1
1,FocuzMindz,"AWS Data Architect/ Engineer with Redshift, RDS","Alexandria, VA",We are seeking an AWS Data Engineer to join ou...,$2L - $2L,#N/A,#N/A,#N/A,#N/A,#N/A,#N/A,1
2,FocuzMindz,"AWS Data Architect/ Engineer with Redshift, RDS","Alexandria, VA",We are seeking an AWS Data Engineer to join ou...,$2L - $2L,#N/A,#N/A,#N/A,#N/A,#N/A,#N/A,1
3,Fiable Consulting,Data and Knowledge Engineer,Remote,"REQUIRED\nPython, Data Pipelines/ETL, Apache N...",$56T - $56T,51 to 200 Employees,Company - Private,Information Technology,Information Technology Support Services,2017,Unknown / Non-Applicable,1
4,Talent Group,Sr. Data Engineer- BHS,Remote,Job Description :\nMust have Behavioral Health...,$1L - $1L,1 to 50 Employees,Company - Public,Media & Communication,Broadcast Media,#N/A,Less than $1 million (USD),1
5,FocuzMindz,"AWS Data Architect/ Engineer with Redshift, RDS","Alexandria, VA",We are seeking an AWS Data Engineer to join ou...,$2L - $2L,#N/A,#N/A,#N/A,#N/A,#N/A,#N/A,1


In [26]:
dados = dados.withColumn('salary_estimate', 
                 when(dados['split_size'] == 1, 
                      concat(dados['salary_estimate'], lit(' - '), dados['salary_estimate'])
                     ).otherwise(dados['salary_estimate'])
                )

In [27]:
dados\
    .withColumn('min_salary', split(dados['salary_estimate'], ' ')[0])\
    .withColumn('max_salary',
                when(split(dados['salary_estimate'], ' ')[1] == 'Per', split(dados['salary_estimate'], ' ')[0])\
                .otherwise(split(dados['salary_estimate'], ' ')[2]))\
    .withColumn("multiplicador", when(dados['salary_estimate'].contains('Per hour'), 'Per hour').otherwise('Per year'))\
    .select('salary_estimate', 'min_salary', 'max_salary')\
    .show()

+--------------------+----------+----------+
|     salary_estimate|min_salary|max_salary|
+--------------------+----------+----------+
|$65.00 - $75.00 P...|    $65.00|    $75.00|
|$90.00 - $120.00 ...|    $90.00|   $120.00|
|$55.00 - $85.00 P...|    $55.00|    $85.00|
|$60.00 - $68.00 P...|    $60.00|    $68.00|
|$45.00 - $50.00 P...|    $45.00|    $50.00|
|           $1L - $2L|       $1L|       $2L|
|$65.00 - $70.00 P...|    $65.00|    $70.00|
|$65.00 - $75.00 P...|    $65.00|    $75.00|
|$50.00 - $80.00 P...|    $50.00|    $80.00|
|          $87T - $1L|      $87T|       $1L|
|$60.00 - $65.00 P...|    $60.00|    $65.00|
|     $73.00 Per hour|    $73.00|    $73.00|
|          $75T - $1L|      $75T|       $1L|
|$60.00 - $65.00 P...|    $60.00|    $65.00|
|           $1L - $3L|       $1L|       $3L|
|           $1L - $2L|       $1L|       $2L|
|           $1L - $1L|       $1L|       $1L|
|          $84T - $1L|      $84T|       $1L|
|          $97T - $1L|      $97T|       $1L|
|$55.00 - 

In [28]:
dados = dados\
    .withColumn('min_salary', split(dados['salary_estimate'], ' ')[0])\
    .withColumn('max_salary',
                when(split(dados['salary_estimate'], ' ')[1] == 'Per', split(dados['salary_estimate'], ' ')[0])\
                .otherwise(split(dados['salary_estimate'], ' ')[2]))\
    .withColumn("multiplicador", when(dados['salary_estimate'].contains('Per hour'), 'Per hour').otherwise('Per year'))

In [29]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import DoubleType

Limpando os caracteres `$`:

In [30]:
dados = dados\
    .withColumn('min_salary_sem', regexp_replace(dados['min_salary'], '^\$', ''))\
    .withColumn('max_salary_sem', regexp_replace(dados['max_salary'], '^\$', ''))\
    .drop('min_salary')\
    .drop('max_salary')

In [31]:
dados.limit(5).toPandas()

,company,job_title,location,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,split_size,multiplicador,min_salary_sem,max_salary_sem
0,DataPattern,Sr. Data Engineer,"Los Angeles, CA",Responsibilities\n● Contribute to the design a...,$65.00 - $75.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,5,Per hour,65.00,75.00
1,YT Global Network,Data Engineer- Remote,Remote,Data Engineer- Remote\nRole: Data and Analytic...,$90.00 - $120.00 Per hour,Unknown,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,5,Per hour,90.00,120.00
2,Wevision LLC,Data Engineer,"Irvine, CA","Job description\nWe build services, data platf...",$55.00 - $85.00 Per hour,1 to 50 Employees,Company - Private,#N/A,#N/A,#N/A,Unknown / Non-Applicable,5,Per hour,55.00,85.00
3,kairos technologies,Data Engineer,"Boston, MA",Job Title: Data Engineer with AWS – Hybrid\nLo...,$60.00 - $68.00 Per hour,51 to 200 Employees,Company - Private,Information Technology,Information Technology Support Services,2003,$5 to $25 million (USD),5,Per hour,60.00,68.00
4,Konnectingtree,Data Engineer,Remote,Greetings from KonnectingTree!\nWe are looking...,$45.00 - $50.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,5,Per hour,45.00,50.00


Aplicando os multiplicadores $L = 10^{5}$ e $T=10^{3}$ nos salários:

In [32]:
dados\
    .withColumn('min_salary',
                when(dados['min_salary_sem'].contains('L'),
                     regexp_replace(dados['min_salary_sem'], 'L', '').cast(DoubleType())*100000)\
                .otherwise(dados['min_salary_sem']))\
    .withColumn('max_salary',
                when(dados['max_salary_sem'].contains('L'),
                     regexp_replace(dados['max_salary_sem'], 'L', '').cast(DoubleType())*100000)\
                .otherwise(dados['max_salary_sem']))\
    .toPandas()

,company,job_title,location,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,split_size,multiplicador,min_salary_sem,max_salary_sem,min_salary,max_salary
0,DataPattern,Sr. Data Engineer,"Los Angeles, CA",Responsibilities\n● Contribute to the design a...,$65.00 - $75.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,5,Per hour,65.00,75.00,65.00,75.00
1,YT Global Network,Data Engineer- Remote,Remote,Data Engineer- Remote\nRole: Data and Analytic...,$90.00 - $120.00 Per hour,Unknown,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,5,Per hour,90.00,120.00,90.00,120.00
2,Wevision LLC,Data Engineer,"Irvine, CA","Job description\nWe build services, data platf...",$55.00 - $85.00 Per hour,1 to 50 Employees,Company - Private,#N/A,#N/A,#N/A,Unknown / Non-Applicable,5,Per hour,55.00,85.00,55.00,85.00
3,kairos technologies,Data Engineer,"Boston, MA",Job Title: Data Engineer with AWS – Hybrid\nLo...,$60.00 - $68.00 Per hour,51 to 200 Employees,Company - Private,Information Technology,Information Technology Support Services,2003,$5 to $25 million (USD),5,Per hour,60.00,68.00,60.00,68.00
4,Konnectingtree,Data Engineer,Remote,Greetings from KonnectingTree!\nWe are looking...,$45.00 - $50.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,5,Per hour,45.00,50.00,45.00,50.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,Konnectingtree,Data Engineer,Remote,Greetings from KonnectingTree!\nWe are looking...,$45.00 - $50.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,5,Per hour,45.00,50.00,45.00,50.00
399,FlexIT Inc,Data Science Engineer,"Beaverton, OR",We are looking for strong experience in Python...,$83T - $1L,1 to 50 Employees,Company - Private,#N/A,#N/A,#N/A,Unknown / Non-Applicable,3,Per year,83T,1L,83T,100000.0
400,Ascendion,Senior Data Engineer,Remote,Responsibilities:\n8+ years professional exper...,$60.00 - $70.00 Per hour,1001 to 5000 Employees,Company - Private,Information Technology,Software Development,2022,Unknown / Non-Applicable,5,Per hour,60.00,70.00,60.00,70.00
401,Findability Sciences,Snowflake Data Engineer pipeline,"Houston, TX",Snowflake data engineers will be responsible f...,$42.14 - $70.29 Per hour,#N/A,#N/A,#N/A,#N/A,#N/A,#N/A,5,Per hour,42.14,70.29,42.14,70.29


In [33]:
dados = dados\
    .withColumn('min_salary',
                when(dados['min_salary_sem'].contains('L'),
                     regexp_replace(dados['min_salary_sem'], 'L', '').cast(DoubleType())*100000)\
                .otherwise(dados['min_salary_sem']))\
    .withColumn('max_salary',
                when(dados['max_salary_sem'].contains('L'),
                     regexp_replace(dados['max_salary_sem'], 'L', '').cast(DoubleType())*100000)\
                .otherwise(dados['max_salary_sem']))\
    .drop('min_salary_sem')

In [34]:
dados = dados\
    .drop('max_salary_sem')\
    .drop('min_salary_sem')\
    .drop('split_size')

In [35]:
dados = dados\
    .withColumn('min_salary_double',
                when(dados['min_salary'].contains('T'),
                     regexp_replace(dados['min_salary'], 'T', '').cast(DoubleType())*1000)\
                .otherwise(dados['min_salary']))\
    .withColumn('max_salary_double',
                when(dados['max_salary'].contains('T'),
                     regexp_replace(dados['max_salary'], 'T', '').cast(DoubleType())*1000)\
                .otherwise(dados['max_salary']))

In [36]:
dados.limit(5).toPandas()

,company,job_title,location,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,multiplicador,min_salary,max_salary,min_salary_double,max_salary_double
0,DataPattern,Sr. Data Engineer,"Los Angeles, CA",Responsibilities\n● Contribute to the design a...,$65.00 - $75.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,Per hour,65.00,75.00,65.00,75.00
1,YT Global Network,Data Engineer- Remote,Remote,Data Engineer- Remote\nRole: Data and Analytic...,$90.00 - $120.00 Per hour,Unknown,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,Per hour,90.00,120.00,90.00,120.00
2,Wevision LLC,Data Engineer,"Irvine, CA","Job description\nWe build services, data platf...",$55.00 - $85.00 Per hour,1 to 50 Employees,Company - Private,#N/A,#N/A,#N/A,Unknown / Non-Applicable,Per hour,55.00,85.00,55.00,85.00
3,kairos technologies,Data Engineer,"Boston, MA",Job Title: Data Engineer with AWS – Hybrid\nLo...,$60.00 - $68.00 Per hour,51 to 200 Employees,Company - Private,Information Technology,Information Technology Support Services,2003,$5 to $25 million (USD),Per hour,60.00,68.00,60.00,68.00
4,Konnectingtree,Data Engineer,Remote,Greetings from KonnectingTree!\nWe are looking...,$45.00 - $50.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,Per hour,45.00,50.00,45.00,50.00


In [37]:
dados = dados\
    .drop('min_salary')\
    .drop('max_salary')

In [38]:
dados.limit(5).toPandas()

,company,job_title,location,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,multiplicador,min_salary_double,max_salary_double
0,DataPattern,Sr. Data Engineer,"Los Angeles, CA",Responsibilities\n● Contribute to the design a...,$65.00 - $75.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,Per hour,65.00,75.00
1,YT Global Network,Data Engineer- Remote,Remote,Data Engineer- Remote\nRole: Data and Analytic...,$90.00 - $120.00 Per hour,Unknown,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,Per hour,90.00,120.00
2,Wevision LLC,Data Engineer,"Irvine, CA","Job description\nWe build services, data platf...",$55.00 - $85.00 Per hour,1 to 50 Employees,Company - Private,#N/A,#N/A,#N/A,Unknown / Non-Applicable,Per hour,55.00,85.00
3,kairos technologies,Data Engineer,"Boston, MA",Job Title: Data Engineer with AWS – Hybrid\nLo...,$60.00 - $68.00 Per hour,51 to 200 Employees,Company - Private,Information Technology,Information Technology Support Services,2003,$5 to $25 million (USD),Per hour,60.00,68.00
4,Konnectingtree,Data Engineer,Remote,Greetings from KonnectingTree!\nWe are looking...,$45.00 - $50.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,Per hour,45.00,50.00


In [39]:
dados = dados\
    .withColumnRenamed('min_salary_double', 'min_salary')\
    .withColumnRenamed('max_salary_double', 'max_salary')\
    .drop('size_split')

In [40]:
dados.limit(5).toPandas()

,company,job_title,location,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,multiplicador,min_salary,max_salary
0,DataPattern,Sr. Data Engineer,"Los Angeles, CA",Responsibilities\n● Contribute to the design a...,$65.00 - $75.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,Per hour,65.00,75.00
1,YT Global Network,Data Engineer- Remote,Remote,Data Engineer- Remote\nRole: Data and Analytic...,$90.00 - $120.00 Per hour,Unknown,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,Per hour,90.00,120.00
2,Wevision LLC,Data Engineer,"Irvine, CA","Job description\nWe build services, data platf...",$55.00 - $85.00 Per hour,1 to 50 Employees,Company - Private,#N/A,#N/A,#N/A,Unknown / Non-Applicable,Per hour,55.00,85.00
3,kairos technologies,Data Engineer,"Boston, MA",Job Title: Data Engineer with AWS – Hybrid\nLo...,$60.00 - $68.00 Per hour,51 to 200 Employees,Company - Private,Information Technology,Information Technology Support Services,2003,$5 to $25 million (USD),Per hour,60.00,68.00
4,Konnectingtree,Data Engineer,Remote,Greetings from KonnectingTree!\nWe are looking...,$45.00 - $50.00 Per hour,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,Per hour,45.00,50.00


In [41]:
dados\
    .select('min_salary')\
    .groupBy('min_salary')\
    .count()\
    .sort(desc('count'))\
    .show(100)

dados\
    .select('max_salary')\
    .groupBy('max_salary')\
    .count()\
    .sort(desc('count'))\
    .show(100)

+----------+-----+
|min_salary|count|
+----------+-----+
|  100000.0|   66|
|     65.00|   32|
|     60.00|   31|
|   90000.0|   21|
|     70.00|   20|
|  200000.0|   17|
|   85000.0|   15|
|   95000.0|   15|
|     42.96|   13|
|   73000.0|   12|
|   86000.0|   12|
|     43.82|   11|
|   83000.0|   11|
|   88000.0|   11|
|     73.00|   10|
|    100.00|   10|
|     55.00|   10|
|     80.00|    9|
|     90.00|    8|
|   70000.0|    7|
|   87000.0|    7|
|     50.00|    6|
|   91000.0|    6|
|     39.86|    5|
|   89000.0|    4|
|   80000.0|    4|
|     45.00|    4|
|   96000.0|    3|
|   75000.0|    3|
|     42.14|    3|
|     52.06|    2|
|     72.00|    2|
|   92000.0|    2|
|   55000.0|    2|
|   39000.0|    1|
|   98000.0|    1|
|   54000.0|    1|
|   82000.0|    1|
|   84000.0|    1|
|   69000.0|    1|
|   97000.0|    1|
|   56000.0|    1|
|   59000.0|    1|
+----------+-----+

+----------+-----+
|max_salary|count|
+----------+-----+
|  100000.0|  140|
|  200000.0|   69|
|     70.00

Convertendo os valores de salário para o tipo **double**:

In [42]:
dados = dados\
    .withColumn('min_salary', dados['min_salary'].cast(DoubleType()))\
    .withColumn('max_salary', dados['max_salary'].cast(DoubleType()))

In [43]:
dados.printSchema()

root
 |-- company: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- location: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- salary_estimate: string (nullable = true)
 |-- company_size: string (nullable = true)
 |-- company_type: string (nullable = true)
 |-- company_sector: string (nullable = true)
 |-- company_industry: string (nullable = true)
 |-- company_founded: string (nullable = true)
 |-- company_revenue: string (nullable = true)
 |-- multiplicador: string (nullable = false)
 |-- min_salary: double (nullable = true)
 |-- max_salary: double (nullable = true)



Cálculo final do salário médio (`salary_avg`):

In [44]:
dados = dados\
    .withColumn('salary_avg', .5*(dados['min_salary'] + dados['max_salary']))\
    .drop('min_salary')\
    .drop('max_salary')\
    .drop('salary_estimate')

Considerando um regime de trabalho de 40h semanais, ou seja 2080 horas trabalhadas no ano, converte-se as médias salarias categorizadas em `'Per hour'`:

In [45]:
dados = dados\
    .withColumn('salary_avg', when(dados['multiplicador'] == 'Per hour', dados['salary_avg']*2080).otherwise(dados['salary_avg']))\
    .drop('multiplicador')

In [46]:
dados.limit(10).toPandas()

,company,job_title,location,job_description,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,salary_avg
0,DataPattern,Sr. Data Engineer,"Los Angeles, CA",Responsibilities\n● Contribute to the design a...,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,145600.0
1,YT Global Network,Data Engineer- Remote,Remote,Data Engineer- Remote\nRole: Data and Analytic...,Unknown,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,218400.0
2,Wevision LLC,Data Engineer,"Irvine, CA","Job description\nWe build services, data platf...",1 to 50 Employees,Company - Private,#N/A,#N/A,#N/A,Unknown / Non-Applicable,145600.0
3,kairos technologies,Data Engineer,"Boston, MA",Job Title: Data Engineer with AWS – Hybrid\nLo...,51 to 200 Employees,Company - Private,Information Technology,Information Technology Support Services,2003,$5 to $25 million (USD),133120.0
4,Konnectingtree,Data Engineer,Remote,Greetings from KonnectingTree!\nWe are looking...,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,98800.0
5,Edmunds.com,Data Engineer,"Santa Monica, CA",Edmunds offers flexibility to work fully remot...,201 to 500 Employees,Company - Private,Information Technology,Internet & Web Services,1966,$100 to $500 million (USD),150000.0
6,Kanini Software Solutions,Data Engineer,Remote,About Kanini\nKanini provides Agile Software D...,501 to 1000 Employees,Company - Private,Information Technology,Enterprise Software & Network Solutions,2003,$5 to $25 million (USD),140400.0
7,Gridiron IT,Data Engineer,"Washington, DC","Seeking a Data Engineer local to Washington, D...",51 to 200 Employees,Company - Private,Information Technology,Information Technology Support Services,2017,Unknown / Non-Applicable,145600.0
8,STAND 8,DATA ENGINEER,Remote,STAND 8 is a global leader providing end-to-en...,1 to 50 Employees,Company - Private,#N/A,#N/A,#N/A,Less than $1 million (USD),135200.0
9,Radiant Systems Inc,Azure Data Engineer,"Milwaukee, WI",Azure Synapse (no data bricks please)\nSome Da...,201 to 500 Employees,Company - Private,Information Technology,Information Technology Support Services,1995,$25 to $50 million (USD),93500.0


In [47]:
dados\
    .select('salary_avg')\
    .groupBy('salary_avg')\
    .count()\
    .sort(desc('count'))\
    .show(5)

+----------+-----+
|salary_avg|count|
+----------+-----+
|  150000.0|   52|
|  145600.0|   30|
|  135200.0|   21|
|   95000.0|   19|
|   92500.0|   15|
+----------+-----+
only showing top 5 rows



### Análise de `'location'`

In [48]:
dados\
    .select('location')\
    .groupBy('location')\
    .count()\
    .sort(desc('count'))\
    .show(100)

+--------------------+-----+
|            location|count|
+--------------------+-----+
|              Remote|   67|
|        Columbus, OH|   26|
|          Dallas, TX|   26|
|        Richmond, VA|   16|
|            Lehi, UT|   16|
|        New York, NY|   13|
|      Washington, DC|   12|
|          Edison, NJ|   12|
|          Boston, MA|   12|
|    Independence, KS|   11|
|   San Francisco, CA|   11|
|     Los Angeles, CA|   11|
|    Florham Park, NJ|   11|
|       Beaverton, OR|   11|
|     Springfield, VA|   11|
|      Chelmsford, MA|   10|
|        O Fallon, MO|    9|
|        Bethesda, MD|    9|
|       Princeton, NJ|    7|
|         Norfolk, VA|    7|
|      Louisville, KY|    7|
|          Newark, NJ|    7|
|  Pleasant Grove, UT|    6|
|    Mount Laurel, NJ|    6|
|      Alexandria, VA|    5|
|      Alpharetta, GA|    5|
|         Seattle, WA|    5|
|         Phoenix, AZ|    4|
|    Philadelphia, PA|    4|
|         Hanover, NJ|    3|
|         Houston, TX|    3|
|        Lakel

Nada a fazer.

### Análise de `'company_size'`:

In [49]:
dados\
    .select('company_size')\
    .groupBy('company_size')\
    .count()\
    .sort(desc('count'))\
    .show(100)

+--------------------+-----+
|        company_size|count|
+--------------------+-----+
|   1 to 50 Employees|  126|
| 51 to 200 Employees|   96|
|201 to 500 Employees|   62|
|                #N/A|   37|
|1001 to 5000 Empl...|   29|
|    10000+ Employees|   15|
|501 to 1000 Emplo...|   15|
|5001 to 10000 Emp...|   12|
|             Unknown|   11|
+--------------------+-----+



In [50]:
dados = dados\
    .withColumn('company_size', when(dados['company_size'] == 'Unknown', '#N/A')\
                .otherwise(dados['company_size']))

In [51]:
dados\
    .select('company_size')\
    .groupBy('company_size')\
    .count()\
    .sort(desc('count'))\
    .show(100)

+--------------------+-----+
|        company_size|count|
+--------------------+-----+
|   1 to 50 Employees|  126|
| 51 to 200 Employees|   96|
|201 to 500 Employees|   62|
|                #N/A|   48|
|1001 to 5000 Empl...|   29|
|    10000+ Employees|   15|
|501 to 1000 Emplo...|   15|
|5001 to 10000 Emp...|   12|
+--------------------+-----+



### Análise de `'company_type'`:

In [52]:
dados\
    .select('company_type')\
    .groupBy('company_type')\
    .count()\
    .sort(desc('count'))\
    .show(truncate=False)

+------------------------------+-----+
|company_type                  |count|
+------------------------------+-----+
|Company - Private             |281  |
|Company - Public              |69   |
|#N/A                          |37   |
|Subsidiary or Business Segment|10   |
|Private Practice / Firm       |4    |
|Contract                      |1    |
|Non-profit Organisation       |1    |
+------------------------------+-----+



### Análise de `'company_sector'`:

In [53]:
dados\
    .select('company_sector')\
    .groupBy('company_sector')\
    .count()\
    .sort(desc('count'))\
    .show(truncate=False)

+------------------------------+-----+
|company_sector                |count|
+------------------------------+-----+
|Information Technology        |205  |
|#N/A                          |112  |
|Finance                       |18   |
|Human Resources & Staffing    |16   |
|Transportation & Logistics    |13   |
|Management & Consulting       |13   |
|Retail & Wholesale            |9    |
|Healthcare                    |7    |
|Pharmaceutical & Biotechnology|3    |
|Manufacturing                 |3    |
|Insurance                     |2    |
|Legal                         |1    |
|Media & Communication         |1    |
+------------------------------+-----+



In [54]:
dados.limit(3).toPandas()

,company,job_title,location,job_description,company_size,company_type,company_sector,company_industry,company_founded,company_revenue,salary_avg
0,DataPattern,Sr. Data Engineer,"Los Angeles, CA",Responsibilities\n● Contribute to the design a...,1 to 50 Employees,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,145600.0
1,YT Global Network,Data Engineer- Remote,Remote,Data Engineer- Remote\nRole: Data and Analytic...,#N/A,Company - Public,#N/A,#N/A,#N/A,Unknown / Non-Applicable,218400.0
2,Wevision LLC,Data Engineer,"Irvine, CA","Job description\nWe build services, data platf...",1 to 50 Employees,Company - Private,#N/A,#N/A,#N/A,Unknown / Non-Applicable,145600.0


### Análise de `'company_industry'`:



In [55]:
dados\
    .select('company_industry')\
    .groupBy('company_industry')\
    .count()\
    .sort(desc('count'))\
    .show(truncate=False)

+---------------------------------------+-----+
|company_industry                       |count|
+---------------------------------------+-----+
|Information Technology Support Services|129  |
|#N/A                                   |112  |
|Computer Hardware Development          |29   |
|Software Development                   |22   |
|Enterprise Software & Network Solutions|21   |
|Investment & Asset Management          |14   |
|Business Consulting                    |13   |
|Shipping & Trucking                    |12   |
|Staffing, Recruitment & Subcontracting |11   |
|Healthcare Services & Hospitals        |7    |
|Automotive Parts & Accessories Stores  |6    |
|HR Consulting                          |5    |
|Internet & Web Services                |4    |
|Biotech & Pharmaceuticals              |3    |
|Grocery Stores                         |2    |
|Banking & Lending                      |2    |
|Accounting & Tax                       |2    |
|Machinery Manufacturing                

### Dados faltantes

[Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions)


In [56]:
import pyspark.sql.functions as f

In [57]:
dados\
    .select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in dados.columns])\
    .show()

+-------+---------+--------+---------------+------------+------------+--------------+----------------+---------------+---------------+----------+
|company|job_title|location|job_description|company_size|company_type|company_sector|company_industry|company_founded|company_revenue|salary_avg|
+-------+---------+--------+---------------+------------+------------+--------------+----------------+---------------+---------------+----------+
|      0|        0|       0|              0|           0|           0|             0|               0|              0|              0|         0|
+-------+---------+--------+---------------+------------+------------+--------------+----------------+---------------+---------------+----------+



## Gerando uma nova feature: `senioridade`

Realiza-se uma varredura pelas colunas `'job_title'` e `'job_description'` para verificar a existência de menções à senioridade da vaga:

In [58]:
from functools import reduce

In [59]:
sr_exp_title = reduce(lambda a, b: a | b, [
    dados['job_title'].rlike("(?i)sr"),
    dados['job_title'].rlike("(?i)senior"),
    dados['job_title'].rlike("(?i)lead"),
    dados['job_title'].rlike("(?i)principal"),
    dados['job_title'].rlike("(?i)manager")
])

jr_exp_title = reduce(lambda a, b: a | b, [
    dados['job_title'].rlike("(?i)jr"),
    dados['job_title'].rlike("(?i)junior"),
    dados['job_title'].rlike("(?i)entry"),
    dados['job_title'].rlike("(?i)associate"),
    dados['job_title'].rlike("(?i)intern")
])

sr_exp_desc = reduce(lambda a, b: a | b, [
    dados['job_description'].rlike("(?i)sr"),
    dados['job_description'].rlike("(?i)senior"),
    dados['job_description'].rlike("(?i)lead"),
    dados['job_description'].rlike("(?i)principal"),
    dados['job_description'].rlike("(?i)manager")
])


jr_exp_desc = reduce(lambda a, b: a | b, [
    dados['job_description'].rlike("(?i)jr"),
    dados['job_description'].rlike("(?i)junior"),
    dados['job_description'].rlike("(?i)entry"),
    dados['job_description'].rlike("(?i)associate"),
    dados['job_description'].rlike("(?i)intern")
])

In [60]:
dados = dados\
    .withColumn('seniority_by_title', when(sr_exp_title, 'Senior').when(jr_exp_title, 'Junior').otherwise(None))\
    .withColumn('seniority_by_desc', when(sr_exp_desc, 'Senior').when(jr_exp_desc, 'Junior').otherwise(None))

In [61]:
dados\
    .select('seniority_by_title')\
    .groupBy('seniority_by_title')\
    .count()\
    .sort(desc('count'))\
    .show(truncate=False)

dados\
    .select('seniority_by_desc')\
    .groupBy('seniority_by_desc')\
    .count()\
    .sort(desc('count'))\
    .show(truncate=False)

+------------------+-----+
|seniority_by_title|count|
+------------------+-----+
|null              |241  |
|Senior            |158  |
|Junior            |4    |
+------------------+-----+

+-----------------+-----+
|seniority_by_desc|count|
+-----------------+-----+
|Senior           |248  |
|null             |136  |
|Junior           |19   |
+-----------------+-----+



**Regras para escolher a informação de senioridade:**

1. Prioriza-se as atribuições da descrição dada pelo *título da vaga* por serem explícitas
2. Quando as senioridades são iguais nas duas colunas, mantém-se-a
3. Quando a senioridade no título é nula e da descrição não, aproveita-se a serionidade capturada na descrição
4. os demais casos são considerados vagas plenas (Mid-level)

In [62]:
dados = dados\
    .withColumn(
        'seniority',
        when(dados['seniority_by_title'].isNotNull(), dados['seniority_by_title'])\
        .when(dados['seniority_by_title'] == dados['seniority_by_desc'], dados['seniority_by_title'])\
        .when(dados['seniority_by_title'].isNull() & dados['seniority_by_desc'].isNotNull(), dados['seniority_by_desc'])\
        .otherwise('Mid'))

In [63]:
dados\
    .select('seniority')\
    .groupBy('seniority')\
    .count()\
    .orderBy(desc('count'))\
    .show()

+---------+-----+
|seniority|count|
+---------+-----+
|   Senior|  276|
|      Mid|  106|
|   Junior|   21|
+---------+-----+



## Gerando uma nova feature: `skills`


Listei algumas ferramentas de Engenharia de Dados muito requisitadas pelo mercado e agreguei a ocorrência delas em `'job_description'` na coluna `'skills'` através de uma lista:


In [64]:
tools = ['AWS',    'GCP',    'Azure',    'Snowflake',    'Databricks',    'Airflow',    'NiFi',    'Talend',    'StreamSets',    'AWS Glue',    'Python',    'SQL',    'Java',    'Scala',    'Bash',    'Spark',    'Hadoop',    'Flink',    'Kafka',    'Presto',    'PostgreSQL',    'MySQL',    'Oracle',    'MongoDB',    'Cassandra',    'Git',    'Jupyter Notebook',    'Tableau',    'Grafana',    'TensorFlow']

tools = [tool.lower() for tool in tools]

In [65]:
from pyspark.sql.functions import array, lower

In [66]:
dados\
    .withColumn('skills', array(*[when(lower(dados['job_description']).contains(tool), 1).otherwise(0) for tool in tools]))\
    .show(5)

+-------------------+--------------------+---------------+--------------------+-------------------+-----------------+--------------------+--------------------+---------------+--------------------+----------+------------------+-----------------+---------+--------------------+
|            company|           job_title|       location|     job_description|       company_size|     company_type|      company_sector|    company_industry|company_founded|     company_revenue|salary_avg|seniority_by_title|seniority_by_desc|seniority|              skills|
+-------------------+--------------------+---------------+--------------------+-------------------+-----------------+--------------------+--------------------+---------------+--------------------+----------+------------------+-----------------+---------+--------------------+
|        DataPattern|   Sr. Data Engineer|Los Angeles, CA|Responsibilities
...|  1 to 50 Employees| Company - Public|                #N/A|                #N/A|           #N

In [67]:
dados = dados\
    .withColumn('skills', array(*[when(lower(dados['job_description']).contains(tool), 1).otherwise(0) for tool in tools]))

## Gerando uma nova feature: `work_mode`


Busca para saber se existe a possibilidade de trabalho remoto:


In [68]:
remote_exp = ['Telecommute',    'Remote',    'Virtual office',    'Teleworking',    'Work from home', 'Distributed team', 'Off site']
onsite_exp = ['On-site',    'In-person',    'Office based',    'Desk job',    'Workplace-based', 'Location-based']

remote_exp = [r.lower().replace('-',' ') for r in remote_exp]
onsite_exp = [o.lower().replace('-',' ') for o in onsite_exp]

In [69]:
dados\
    .withColumn('remote0', array(*[when(lower(dados['job_description']).contains(r), 1).otherwise(0) for r in remote_exp]))\
    .withColumn('onsite0', array(*[when(lower(dados['job_description']).contains(o), 1).otherwise(0) for o in onsite_exp]))\
    .show(5)

+-------------------+--------------------+---------------+--------------------+-------------------+-----------------+--------------------+--------------------+---------------+--------------------+----------+------------------+-----------------+---------+--------------------+--------------------+------------------+
|            company|           job_title|       location|     job_description|       company_size|     company_type|      company_sector|    company_industry|company_founded|     company_revenue|salary_avg|seniority_by_title|seniority_by_desc|seniority|              skills|             remote0|           onsite0|
+-------------------+--------------------+---------------+--------------------+-------------------+-----------------+--------------------+--------------------+---------------+--------------------+----------+------------------+-----------------+---------+--------------------+--------------------+------------------+
|        DataPattern|   Sr. Data Engineer|Los Angele

In [70]:
dados = dados\
    .withColumn('remote0', array(*[when(lower(dados['job_description']).contains(r), 1).otherwise(0) for r in remote_exp]))\
    .withColumn('onsite0', array(*[when(lower(dados['job_description']).contains(o), 1).otherwise(0) for o in onsite_exp]))

In [71]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def bin_reduce(mode_list):
    return reduce(lambda a, b: a|b, mode_list)

bin_reduce_udf = udf(bin_reduce, IntegerType())

In [72]:
dados\
    .withColumn('remote0', bin_reduce_udf(dados['remote0']))\
    .withColumn('onsite0', bin_reduce_udf(dados['onsite0']))\
    .show(5)

+-------------------+--------------------+---------------+--------------------+-------------------+-----------------+--------------------+--------------------+---------------+--------------------+----------+------------------+-----------------+---------+--------------------+-------+-------+
|            company|           job_title|       location|     job_description|       company_size|     company_type|      company_sector|    company_industry|company_founded|     company_revenue|salary_avg|seniority_by_title|seniority_by_desc|seniority|              skills|remote0|onsite0|
+-------------------+--------------------+---------------+--------------------+-------------------+-----------------+--------------------+--------------------+---------------+--------------------+----------+------------------+-----------------+---------+--------------------+-------+-------+
|        DataPattern|   Sr. Data Engineer|Los Angeles, CA|Responsibilities
...|  1 to 50 Employees| Company - Public|       

In [73]:
dados = dados\
    .withColumn('remote0', bin_reduce_udf(dados['remote0']))\
    .withColumn('onsite0', bin_reduce_udf(dados['onsite0']))

In [74]:
dados\
    .withColumn('remote', when(dados['location'] == 'Remote', 1).otherwise(dados['remote0']))\
    .withColumn('onsite', when(dados['location'] == 'Remote', 0).otherwise(dados['onsite0']))\
    .show(5)

+-------------------+--------------------+---------------+--------------------+-------------------+-----------------+--------------------+--------------------+---------------+--------------------+----------+------------------+-----------------+---------+--------------------+-------+-------+------+------+
|            company|           job_title|       location|     job_description|       company_size|     company_type|      company_sector|    company_industry|company_founded|     company_revenue|salary_avg|seniority_by_title|seniority_by_desc|seniority|              skills|remote0|onsite0|remote|onsite|
+-------------------+--------------------+---------------+--------------------+-------------------+-----------------+--------------------+--------------------+---------------+--------------------+----------+------------------+-----------------+---------+--------------------+-------+-------+------+------+
|        DataPattern|   Sr. Data Engineer|Los Angeles, CA|Responsibilities
...|  1

Tratativa para apenas remoto:

In [75]:
dados = dados\
    .withColumn('remote', when(dados['location'] == 'Remote', 1).otherwise(dados['remote0']))\
    .withColumn('onsite', when(dados['location'] == 'Remote', 0).otherwise(dados['onsite0']))\
    .drop('remote0')\
    .drop('onsite0')

In [76]:
dados.show()

+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+----------+------------------+-----------------+---------+--------------------+------+------+
|             company|           job_title|        location|     job_description|        company_size|        company_type|      company_sector|    company_industry|company_founded|     company_revenue|salary_avg|seniority_by_title|seniority_by_desc|seniority|              skills|remote|onsite|
+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+----------+------------------+-----------------+---------+--------------------+------+------+
|         DataPattern|   Sr. Data Engineer| Los Angeles, CA|Responsibilities
...|   1 to 50 Employees|    Compan

## Disponibilização do dataset em parquet

In [101]:
dados.write.mode('overwrite').parquet('/content/previsao_remuneracao/tratamento/dados_parquet')

In [102]:
!pwd

/content


In [116]:
!ls

previsao_remuneracao  spark-3.1.2-bin-hadoop2.7
sample_data	      spark-3.1.2-bin-hadoop2.7.tgz


In [130]:
%cd /content/previsao_remuneracao/

/content/previsao_remuneracao


In [131]:
!pwd

/content/previsao_remuneracao


In [132]:
!ls

airflow_script	  notebook_scrapping  tratamento
machine_learning  README.md	      venv


In [151]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   tratamento/dados_parquet/._SUCCESS.crc
	new file:   tratamento/dados_parquet/.part-00000-c7d82ceb-8664-42d7-8fcf-3bbd78e4bd7c-c000.snappy.parquet.crc
	new file:   tratamento/dados_parquet/_SUCCESS
	new file:   tratamento/dados_parquet/part-00000-c7d82ceb-8664-42d7-8fcf-3bbd78e4bd7c-c000.snappy.parquet



In [152]:
!git add .

In [158]:
!git commit -m "disponibilizacao do dataset para ml em parquet 1"

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [160]:
!git push https://edinaldoab:<token>/edinaldoab/previsao_remuneracao

Everything up-to-date
